# Classifier Tests with extracted data

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import extractor as ext

In [2]:
fbase = pd.read_csv('./1_filtered.csv', header=None)
fbase.head()

,0,1,2,3
0,8.456798,-4.466378,7.215951,0.019221
1,8.454815,-4.464132,7.212556,0.019295
2,8.453542,-4.461002,7.207107,0.019680
3,8.449433,-4.457170,7.201194,0.020556
4,8.440568,-4.453560,7.195996,0.020409


In [3]:
extractor = ext.RecordExtractor()
cycles = extractor.read_sample(fbase)
len(cycles)

4

In [4]:
# First Sensor, First cycle
cycles[0][0].grip.head()

108000    0.037666
108001    0.012989
108002   -0.025789
108003   -0.062451
108004   -0.078099
Name: 0, dtype: float64

In [5]:
# First Sensor, First cycle
cycles[0][0].flexion.head()

48000   -0.076478
48001   -0.076115
48002   -0.048953
48003   -0.003499
48004    0.039958
Name: 0, dtype: float64

# Generate Dataset based on feature extractors

1. SampEn - Sample Entropy
2. RMS - Root Mean Square
3. WL - Waveform Length
4. WAMP - Willison Amplitude
5. ApEn - Approximate entropy
6. MAV - Mean Absolute Value

The data will be offsetted with 100ms.
Each cycle will be interpreted as a new data acquisition
3 Datasets, one with 299ms, other with 500ms, other with 1 second.

Dataset 299ms:
| SENSOR_0_RMS   |  SENSOR_1_RMS | SENSOR_2_RMS | SENSOR_3_RMS| SENSOR_0_SAMPEN   |  SENSOR_1_SAMPEN | SENSOR_2_SAMPEN | SENSOR_3_SAMPEN |...| CLASS | 
| --- | ---| ---| ---| ---| ---| ---| ---| ---| ---|
| 12 | 13 | 45 | 4 | 5 | 23 | 123 | 123|...|GRIP|


TODO: Do we use the onset algorithm?

IDEIA:

Usar dois dados rápidos para análise rápida e depois no meio do movimento da mão verificar a análise com mais tempo e corrigir o movimento

In [8]:
extractor = ext.RecordExtractor()
cycles = extractor.read_sample(fbase)
np.array(cycles).shape

(4, 5)

In [9]:
import feature_extractors as fe

In [36]:
def populatedict(data, name, dic, classs):
    data = np.array(data)
    if not dic.get(name+"_RMS"): dic[name+"_RMS"] = []
    if not dic.get(name+"_WAVELEN"): dic[name+"_WAVELEN"] = []
    if not dic.get(name+"_WAMP"): dic[name+"_WAMP"] = []
    if not dic.get(name+"_APPEN"): dic[name+"_APPEN"] = []
    if not dic.get(name+"_SAMPEN"): dic[name+"_SAMPEN"] = []
    if not dic.get(name+"_MAV"): dic[name+"_MAV"] = []
    if not dic.get("class"): dic["class"] = []

    dic[name+"_RMS"].append(fe.rms(data))
    dic[name+"_WAVELEN"].append(fe.waveformlen(data))
    dic[name+"_WAMP"].append(fe.wamp(data))
    dic[name+"_APPEN"].append(fe.app_entropy(data))
    dic[name+"_SAMPEN"].append(fe.sampen(data))
    dic[name+"_MAV"].append(fe.mav(data))

    dic["class"].append(classs)

    print(len(dic[name+"_RMS"]))
    print(len(dic[name+"_WAVELEN"]))
    print(len(dic["class"]))


In [37]:
newds = {}
newds_offsetted = {}

for j in range(4):
    for i in range(5):
        lower_limit = int(extractor.to_ms(100))
        upper_limit = int(extractor.to_ms(299)) # TODO: Create two databases, one with the offset and the other without

        populatedict(cycles[j][i].rest[:upper_limit], "SENSOR" + str(j), newds, "rest")
        populatedict(cycles[j][i].rest[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted, "rest")

        populatedict(cycles[j][i].extension[:upper_limit], "SENSOR" + str(j), newds, "extension")
        populatedict(cycles[j][i].extension[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted, "extension")

        populatedict(cycles[j][i].flexion[:upper_limit], "SENSOR" + str(j), newds, "flexion")
        populatedict(cycles[j][i].flexion[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted, "flexion")

        populatedict(cycles[j][i].ulnar_deviation[:upper_limit], "SENSOR" + str(j), newds, "ulnar_deviation")
        populatedict(cycles[j][i].ulnar_deviation[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted, "ulnar_deviation")

        populatedict(cycles[j][i].radial_deviation[:upper_limit], "SENSOR" + str(j), newds, "radial_deviation")
        populatedict(cycles[j][i].radial_deviation[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted, "radial_deviation")

        populatedict(cycles[j][i].grip[:upper_limit], "SENSOR" + str(j), newds, "grip")
        populatedict(cycles[j][i].grip[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted, "grip")

        populatedict(cycles[j][i].finger_abduction[:upper_limit], "SENSOR" + str(j), newds, "finger_abduction")
        populatedict(cycles[j][i].finger_abduction[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted, "finger_abduction")

        populatedict(cycles[j][i].finger_adduction[:upper_limit], "SENSOR" + str(j), newds, "finger_adduction")
        populatedict(cycles[j][i].finger_adduction[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted, "finger_adduction")

        populatedict(cycles[j][i].supination[:upper_limit], "SENSOR" + str(j), newds, "supination")
        populatedict(cycles[j][i].supination[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted, "supination")

        populatedict(cycles[j][i].pronation[:upper_limit], "SENSOR" + str(j), newds, "pronation")
        populatedict(cycles[j][i].pronation[lower_limit:lower_limit+upper_limit], "SENSOR" + str(j), newds_offsetted, "pronation")

len(newds)

1
1
1
1
1
1
2
2
2
2
2
2
3
3
3
3
3
3
4
4
4
4
4
4
5
5
5
5
5
5
6
6
6
6
6
6
7
7
7
7
7
7
8
8
8
8
8
8
9
9
9
9
9
9
10
10
10
10
10
10
11
11
11
11
11
11
12
12
12
12
12
12
13
13
13
13
13
13
14
14
14
14
14
14
15
15
15
15
15
15
16
16
16
16
16
16
17
17
17
17
17
17
18
18
18
18
18
18
19
19
19
19
19
19
20
20
20
20
20
20
21
21
21
21
21
21
22
22
22
22
22
22
23
23
23
23
23
23
24
24
24
24
24
24
25
25
25
25
25
25
26
26
26
26
26
26
27
27
27
27
27
27
28
28
28
28
28
28
29
29
29
29
29
29
30
30
30
30
30
30
31
31
31
31
31
31
32
32
32
32
32
32
33
33
33
33
33
33
34
34
34
34
34
34
35
35
35
35
35
35
36
36
36
36
36
36
37
37
37
37
37
37
38
38
38
38
38
38
39
39
39
39
39
39
40
40
40
40
40
40
41
41
41
41
41
41
42
42
42
42
42
42
43
43
43
43
43
43
44
44
44
44
44
44
45
45
45
45
45
45
46
46
46
46
46
46
47
47
47
47
47
47
48
48
48
48
48
48
49
49
49
49
49
49
50
50
50
50
50
50
1
1
51
1
1
51
2
2
52
2
2
52
3
3
53
3
3
53
4
4
54
4
4
54
5
5
55
5
5
55
6
6
56
6
6
56
7
7
57
7
7
57
8
8
58
8
8
58
9
9
59
9
9
59
10
10
60
10
10
60
11
11
61
1

25

In [31]:
newds

{'SENSOR0_RMS': [0.03172136304971984,
  0.07054778351194317,
  0.027455197804295867,
  0.029670459885460652,
  0.02763249512758615,
  0.025400845821693593,
  0.03122827606440986,
  0.04074809736449961,
  0.025068416001111676,
  0.023237803423570635,
  0.027660760495710555,
  0.1497775793574657,
  0.019102369463718634,
  0.032883303232615776,
  0.02279864784507097,
  0.03847429199381519,
  0.024265322753421494,
  0.031804070579302055,
  0.030610923017998164,
  0.021352085726286437,
  0.028181062148739243,
  0.16946740148737163,
  0.017611903725250677,
  0.037038465498519156,
  0.025711548786647546,
  0.025687455548255786,
  0.024930082012246987,
  0.036245602159368354,
  0.030970691301359772,
  0.02274646522069594,
  0.030377904033636702,
  0.1441695262949121,
  0.014849040312886124,
  0.03371710332956996,
  0.0285058256037138,
  0.026546140080249362,
  0.030880789302934142,
  0.04188943513240076,
  0.03248719795612174,
  0.021433269709118363,
  0.028181062148739243,
  0.169467401487371

In [29]:
pd.DataFrame(newds)

ValueError: All arrays must be of the same length

## Sample Entropy

In [ ]:
fe.sampen

In [ ]:
import numpy as np

import sklearn as sk

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error